**Construyendo un proyecto de aprendizaje de máquina**

*Códigos referencia del libro Hands-On Machine Learning with Scikit-Learn and Tensor Flow 2017 (Cap 2) -- Aurélien Géron

# Etapas principales en un sistema de aprendizaje de máquina

1. Observar el problema (ayuda interdisciplinar)
2. Obtener los datos
3. Análisis exploratorio incial (visualizar los datos y estadística descriptiva básica)
4. Preparar los datos para los algoritmos de aprendizaje de máquina (evaluación, preproceso, caracterización, aprendizaje)
5. Seleccionar un modelo y entrenar
6. Sintonizar el modelo escogido
7. Presentar la solución
8. Lanzar la solución, monitorear y mantener el sistema de aprendizaje de máquina.

# Problema a resolver

*Objetivo: predecir precios medios en distritos de California-USA.*

*Insumos: Características de los distritos*

# Inicio códigos

**Preparar modulos principales, funciones inline, paths para guardar archivos y figuras:**

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "imagesAM", CHAPTER_ID)
HOUSING_PATH = "datasets/housing/"

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Obtener los datos


In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()
housing.head(10)

# Análisis exploratorio básico

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
save_fig("attribute_histogram_plots")
plt.show()

# Preparar datos (validación y análisis exploratorio por visualización)

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
housing["median_income"].hist()

**Preproceso variable median_income -> continua a categórica**

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
housing["income_cat"].value_counts()

In [ ]:
housing["income_cat"].hist()


**Muestreo estratificado de datos: asegurar mismas proporciones en las particiones de datos con base a característica de interés**

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"].values):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_train_set["income_cat"].value_counts() / len(strat_train_set)

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
housing["income_cat"].value_counts() / len(housing)

**Comparar errores en particiones con y sin estratificación**

In [ ]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
compare_props

In [ ]:
for set_ in (strat_train_set, strat_test_set): #remove rows or columns
    set_.drop("income_cat", axis=1, inplace=True)

# Análisis exploratorio y visualización sobre datos muestreados

In [ ]:
housing = strat_train_set.copy()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")
save_fig("bad_visualization_plot")

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.2)
save_fig("better_visualization_plot")

The argument `sharex=False` fixes a display bug (the x-axis values and legend were not displayed). This is a temporary fix (see: https://github.com/pandas-dev/pandas/issues/10611). Thanks to Wilmer Arellano for pointing it out.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()
save_fig("housing_prices_scatterplot")

In [ ]:
import matplotlib.image as mpimg
california_img=mpimg.imread(PROJECT_ROOT_DIR + '/imagesAM/end_to_end_project/california.png')
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                       s=housing['population']/100, label="Population",
                       c="median_house_value", cmap=plt.get_cmap("jet"),
                       colorbar=False, alpha=0.4,
                      )
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar()
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
save_fig("california_housing_prices_plot")
plt.show()

**Identificación de correlaciones**

In [ ]:
corr_matrix = housing.corr()
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
import seaborn as sns
sns.heatmap(corr_matrix,xticklabels=corr_matrix.columns.values,yticklabels=corr_matrix.columns.values)
save_fig("corr_matrix")

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# from pandas.tools.plotting import scatter_matrix # For older versions of Pandas
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
save_fig("scatter_matrix_plot")

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])
save_fig("income_vs_house_value_scatterplot")

**Generar nuevas características intuitivas**

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing.plot(kind="scatter", x="rooms_per_household", y="median_house_value",
             alpha=0.2)
plt.axis([0, 5, 0, 520000])
plt.show()

In [ ]:
housing.describe()

# Preparar los datos para los algoritmos de aprendizaje de máquina

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

# Opciones básicas para lidiar con datos perdidos

1. Quitar filas (instancias con característica perdida)
2. Quitar columnas (se elimina la característica completa si presenta datos perdidos)
3. Se estiman los valores perdidos mediante mediana, promedio, moda, o estimaciones por vecindario.

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # option 1

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)       # option 2

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # option 3
sample_incomplete_rows

**Clase Imputer de scikit-learn contiene este preproceso para datos perdidos**

In [ ]:
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer

imputer = SimpleImputer(strategy="median")

Se deben remover las variables nominales para aplicar la opción por mediana:

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)
# alternatively: housing_num = housing.select_dtypes(include=[np.number])

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_ #valores obtenidos en preproceso imputer por atributo numérico

Dichos valores son las mismas medianas:

In [ ]:
housing_num.median().values

Aplicar el preproceso (transformación) según lo entrenado con imputer:

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index = list(housing.index.values))

In [ ]:
housing_tr.loc[sample_incomplete_rows.index.values]

In [ ]:
imputer.strategy

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
housing_tr.head()

# Cómo trabajar con variables nominales (categóricas)? (ej: 'ocean_proximity')

1. Etiquetar (codificar) los textos. Problemas: la noción de cercanía pierde sentido
2. One-hot-encoding, una nueva característica binaria se genera por cada categoría de la variable de interés. Problema: el número de características crece considerablemente.

In [ ]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder
except ImportError:
    from future_encoders import OrdinalEncoder # Scikit-Learn < 0.20

In [ ]:
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

Por defecto, la clase `OneHotEncoder` retorna un arreglo ralo (sparse), se puede usar la función `toarray()` para trabajar con arreglos completos (full).

In [ ]:
housing_cat_1hot.toarray()

También se puede fijar la opción  `sparse=False` en `OneHotEncoder`:

In [ ]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

Let's create a custom transformer to add extra attributes:

In [ ]:
from sklearn.preprocessing import FunctionTransformer

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]


def add_extra_features(X, add_bedrooms_per_room=True):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
        return np.c_[X, rooms_per_household, population_per_household,
                     bedrooms_per_room]
    else:
        return np.c_[X, rooms_per_household, population_per_household]

attr_adder = FunctionTransformer(add_extra_features, validate=False,
                                 kw_args={"add_bedrooms_per_room": False})
housing_extra_attribs = attr_adder.fit_transform(housing.values)

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"])
housing_extra_attribs.head()

# Esquema general (pipeline) de preproceso para atributos numéricos

1. Lidiar con datos perdidos -> SimpleImputer
2. Añadir nuevos atributos desde conocimiento a priori (equipo interdisciplinar) -> FunctionTransformer
3. Escalado de atributos (min max ; estandarización - zscore). Min-max sensible a atípicos  

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()), #MinMaxScaler StandardScaler
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_labels=stats.zscore(housing_labels)

Un esquema general, incluyendo atributos numéricos y categóricos se puede generar con la clase `ColumnTransformer`

In [ ]:
try:
    from sklearn.compose import ColumnTransformer
except ImportError:
    from future_encoders import ColumnTransformer # Scikit-Learn < 0.20

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs), #OneHotEncoder or OrdinalEncoder()
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape

# Seleccionar un modelo y entrenar sobre los datos preparados,Sintonizar parametros

# Regresión lineal

La regresión lineal es una técnica de análisis predictivo básica que utiliza datos históricos para predecir una variable de salida

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
from sklearn.model_selection import cross_val_score

scoreslin_reg = cross_val_score(lin_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=5) #scikitlearn trabaja con función util (mayor mejor) no función de costo (menor mejor)
tree_rmse_scores_lin_reg = np.sqrt(-scoreslin_reg)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores_lin_reg)

In [ ]:
title = "Curvas de aprendizaje "
plt.figure()
plt.title(title)
plt.xlabel("Training examples")
plt.ylabel("Score")

#Validación cruzada para pintar las curvas de entrenamiento y validación e incertidumbre   
train_sizes_lin_reg, train_scores_lin_reg, test_scores_lin_reg= \
    learning_curve(lin_reg, housing_prepared, housing_labels,
                   scoring="neg_mean_squared_error", cv=5)

train_scores_mean_lin_reg= abs(np.mean(train_scores_lin_reg, axis=1))
train_scores_std_lin_reg = abs(np.std(train_scores_lin_reg, axis=1))
test_scores_mean_lin_reg =abs( np.mean(test_scores_lin_reg, axis=1))
test_scores_std_lin_reg =abs( np.std(test_scores_lin_reg, axis=1))


plt.grid()
plt.fill_between(train_sizes_lin_reg, train_scores_mean_lin_reg - train_scores_std_lin_reg,
                 train_scores_mean_lin_reg + train_scores_std_lin_reg, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes_lin_reg, test_scores_mean_lin_reg - test_scores_std_lin_reg,
                 test_scores_mean_lin_reg + test_scores_std_lin_reg, alpha=0.1, color="g")

plt.plot(train_sizes_lin_reg, train_scores_mean_lin_reg, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes_lin_reg, test_scores_mean_lin_reg, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")


# Regresión de Ridge 

Regresión de Ridge La regresión lineal regular tiene la forma de: J (theta) = MSE (theta) La regresión de Ridge aplica un término de regularización proporcional al cuadrado de la norma l2 de los pesos de las características (sin incluir la intersección). 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

param_grid_ridgeReg={ "alpha": [0.1,1,60,200, 1000]}

ridgeReg = Ridge(alpha=10)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search_ridgeReg = GridSearchCV(ridgeReg, param_grid_ridgeReg, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)

grid_search_ridgeReg.fit(housing_prepared, housing_labels)


In [ ]:
cvres_ridgeReg = grid_search_ridgeReg.cv_results_    
testscores_mean_ridgeReg =abs(cvres_ridgeReg["mean_test_score"])
testscores_sd_ridgeReg = abs(cvres_ridgeReg['std_test_score'])

trainscores_mean_ridgeReg =abs(cvres_ridgeReg['mean_train_score'])
trainscores_sd_ridgeReg = abs(cvres_ridgeReg[ 'std_train_score'])



X_axis_ridgeReg = np.array(cvres_ridgeReg['param_alpha'].data, dtype=float)
plt.fill_between(X_axis_ridgeReg, testscores_mean_ridgeReg - testscores_sd_ridgeReg,
                        testscores_mean_ridgeReg + testscores_sd_ridgeReg,alpha=0.1,color="r")    
plt.plot(X_axis_ridgeReg, testscores_mean_ridgeReg, 'o-', color="r",
         label="Training score")

plt.fill_between(X_axis_ridgeReg, trainscores_mean_ridgeReg - trainscores_sd_ridgeReg,
                 trainscores_mean_ridgeReg + trainscores_sd_ridgeReg, alpha=0.1, color="g")

plt.plot(X_axis_ridgeReg, trainscores_mean_ridgeReg, 'o-', color="g",
         label="Cross-validation score")

X_best_params_ridgeReg=grid_search_ridgeReg.best_params_
X_alpha_ridgeReg = cvres_ridgeReg['param_alpha'].data
plt.plot([X_best_params_ridgeReg.get("alpha"), ] * 2, [0.3, abs(testscores_mean_ridgeReg[np.where(X_alpha_ridgeReg == X_best_params_ridgeReg.get("alpha"))])],
         linestyle='-.', color='k', marker='x', markeredgewidth=2, ms=8)

plt.annotate("%0.2f" % abs(testscores_mean_ridgeReg[np.where(X_alpha_ridgeReg == X_best_params_ridgeReg.get("alpha") )]),
             (X_best_params_ridgeReg.get("alpha") , abs(testscores_mean_ridgeReg[np.where(X_alpha_ridgeReg == X_best_params_ridgeReg.get("alpha") )]+ 0.005)))

plt.grid()
plt.legend(loc="best") 
title = "Curvas de aprendizaje "
plt.title(title)
plt.xlabel("Training examples")
plt.ylabel("Score")

Los mejores hyperparámetros encontrados:

In [ ]:
grid_search_ridgeReg.best_params_

In [ ]:
grid_search_ridgeReg.best_estimator_

In [ ]:
for mean_score, params in zip(cvres_ridgeReg["mean_test_score"], cvres_ridgeReg["params"]):
    print(np.sqrt(-mean_score), params)

# Lasso Regression
La ventaja de la regresión de Lazo sobre la rigida se encuentra en la forma de diamante del contorno de la penalización de la norma l1, lo que hace que algunas de las tetas se eliminen (se establezcan en 0) rápidamente. Esto significa que la regresión Lasso puede realizar la selección automática de características, cuando la regresión rigida no puede.

In [ ]:
from sklearn.linear_model import Lasso

param_grid_Lasso={ "alpha": [0.001,0.009,0.02,0.3,2,4]}
ridLasso = Lasso(alpha=2)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search_ridLasso = GridSearchCV(ridLasso, param_grid_Lasso, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search_ridLasso.fit(housing_prepared, housing_labels)

In [ ]:
from sklearn.linear_model import Lasso

cvres_ridLasso = grid_search_ridLasso.cv_results_    
testscores_mean_ridLasso =abs(cvres_ridLasso["mean_test_score"])
testscores_sd_ridLasso = abs(cvres_ridLasso['std_test_score'])

trainscores_mean_ridLasso =abs(cvres_ridLasso['mean_train_score'])
trainscores_sd_ridLasso = abs(cvres_ridLasso[ 'std_train_score'])

X_axis_ridLasso = np.array(cvres_ridLasso['param_alpha'].data, dtype=float)

plt.fill_between(X_axis_ridLasso, testscores_mean_ridLasso - testscores_sd_ridLasso,
                        testscores_mean_ridLasso + testscores_sd_ridLasso,alpha=0.1,color="r")   

plt.plot(X_axis_ridLasso, testscores_mean_ridLasso, 'o-', color="r",
         label="Training score")

plt.fill_between(X_axis_ridLasso, trainscores_mean_ridLasso - trainscores_sd_ridLasso,
                 trainscores_mean_ridLasso + trainscores_sd_ridLasso, alpha=0.1, color="g")

plt.plot(X_axis_ridLasso, trainscores_mean_ridLasso, 'o-', color="g",
         label="Cross-validation score")

X_best_params_ridLasso=grid_search_ridLasso.best_params_

X_alpha_ridLasso = cvres_ridLasso['param_alpha'].data

plt.plot([X_best_params_ridLasso.get("alpha"), ] * 2, [0, abs(testscores_mean_ridLasso[np.where(X_alpha_ridLasso == X_best_params_ridLasso.get("alpha"))])],
         linestyle='-.', color='k', marker='x', markeredgewidth=2, ms=8)

plt.annotate("%0.2f" % abs(testscores_mean_ridLasso[np.where(X_alpha_ridLasso == X_best_params_ridLasso.get("alpha"))]),
             (X_best_params_ridLasso.get("alpha"), abs(testscores_mean_ridLasso[np.where(X_alpha_ridLasso == X_best_params_ridLasso.get("alpha"))]+ 0.005)))

plt.legend(loc="best") 


plt.grid()
plt.legend(loc="best") 
title = "Curvas de aprendizaje "
plt.title(title)
plt.xlabel("Training examples")
plt.ylabel("Score")


Los mejores hyperparámetros encontrados:

In [ ]:
grid_search_ridLasso.best_params_

In [ ]:
grid_search_ridLasso.best_estimator_

In [ ]:
for mean_score, params in zip(cvres_ridLasso["mean_test_score"], cvres_ridLasso["params"]):
    print(np.sqrt(-mean_score), params)

# Elastic Net
La red elástica está en algún lugar entre la regresión de la cresta y la regresión del lazo

In [ ]:
from sklearn.linear_model import ElasticNet

alpha = np.array([0.0001,0.1,10])
l1_ratio = np.array([2,20,50])

param_grid_ElasticNet= [
    # try 12 (3×4) combinations of hyperparameters
    {"alpha": alpha,"l1_ratio": l1_ratio}
  ]


ridElastic = ElasticNet(alpha = 0.1, l1_ratio =0.1)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search_ridElastic = GridSearchCV(ridElastic, param_grid_ElasticNet, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search_ridElastic.fit(housing_prepared, housing_labels)

In [ ]:
cvres_ridElastic = grid_search_ridElastic.cv_results_ 

X_best_params_ridElastic=grid_search_ridElastic.best_params_
testscores_mean_ridElastic = abs(cvres_ridElastic["mean_test_score"])
testscores_mean_ridElastic = np.array(testscores_mean_ridElastic).reshape(len(l1_ratio),len(alpha))

testscores_sd_ridElastic  =  abs(cvres_ridElastic['std_test_score'])
testscores_sd_ridElastic = np.array(testscores_sd_ridElastic).reshape(len(l1_ratio),len(alpha))

# Plot Grid search scores
_, ax = plt.subplots(1,1)

# Param1 is the X-axis, Param 2 is represented as a different curve (color line)"l1_ratio" "alpha"
for idx, val in enumerate(alpha):
    ax.plot(l1_ratio, testscores_mean_ridElastic[idx,:], '-o', label= 'alpha' + ': ' + str(val)) 

ax.plot([X_best_params_ridElastic.get("l1_ratio"), ] * 2, [0, abs(testscores_mean_ridElastic[np.where(alpha == X_best_params_ridElastic.get("alpha")),np.where(l1_ratio == X_best_params_ridElastic.get("l1_ratio"))])],
        linestyle='-.', color='k', marker='x', markeredgewidth=2, ms=8)  
ax.grid()
ax.set_title("Grid Search Scores", fontsize=20, fontweight='bold')
ax.set_xlabel('alpha', fontsize=16)
ax.set_ylabel('CV Average Score', fontsize=16)
ax.legend(loc="best", fontsize=15)
ax.annotate("%0.4f" % abs(testscores_mean_ridElastic[np.where(alpha == X_best_params_ridElastic.get("alpha")),np.where(l1_ratio == X_best_params_ridElastic.get("l1_ratio"))]),
             (X_best_params_ridElastic.get("l1_ratio"), abs(testscores_mean_ridElastic[np.where(alpha == X_best_params_ridElastic.get("alpha")),np.where(l1_ratio == X_best_params_ridElastic.get("l1_ratio"))] + 0.005)))

In [ ]:
grid_search_ridElastic.best_params_

In [ ]:
grid_search_ridElastic.best_estimator_

In [ ]:
for mean_score, params in zip(cvres_ridElastic["mean_test_score"],cvres_ridElastic["params"]):
    print(np.sqrt(-mean_score), params)  

# KernelRidge rbf

combina Kernel ridge regression(mínimos cuadrados lineales con la regularización de la norma l2) con el truco del kernel. Así aprende una función lineal en el espacio inducido por el kernel respectivo y los datos. Para los núcleos no lineales, esto corresponde a una función no lineal en el espacio original.

Nota: al tener tantos dato el proceso podría tardar un tiempo considerable si no se paraleliza

In [ ]:
from sklearn.kernel_ridge import KernelRidge

gamma = np.array([0.00001,0.0001,0.01,0.1])
param_grid_kernel = [ {"gamma": gamma}]

grid_search_KernelRidge= GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1),param_grid_kernel, cv=5,
                                         scoring='neg_mean_squared_error', return_train_score=True)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search_KernelRidge.fit(housing_prepared[:6500,:], housing_labels[:6500])


In [ ]:
cvres_KernelRidge = grid_search_KernelRidge.cv_results_     
testscores_mean_KernelRidge =abs(cvres_KernelRidge["mean_test_score"])
testscores_sd_KernelRidge = abs(cvres_KernelRidge['std_test_score'])

trainscores_mean_KernelRidge =abs(cvres_KernelRidge['mean_train_score'])
trainscores_sd_KernelRidge = abs(cvres_KernelRidge[ 'std_train_score'])



X_axis_KernelRidge = np.array(cvres_KernelRidge['param_gamma'].data, dtype=float)
plt.fill_between(X_axis_KernelRidge, testscores_mean_KernelRidge - testscores_sd_KernelRidge,
                        testscores_mean_KernelRidge + testscores_sd_KernelRidge,alpha=0.1,color="r")    
plt.plot(X_axis_KernelRidge, testscores_mean_KernelRidge, 'o-', color="r",
         label="Training score")

plt.fill_between(X_axis_KernelRidge, trainscores_mean_KernelRidge - trainscores_sd_KernelRidge,
                 trainscores_mean_KernelRidge + trainscores_sd_KernelRidge, alpha=0.1, color="g")

plt.plot(X_axis_KernelRidge, trainscores_mean_KernelRidge, 'o-', color="g",
         label="Cross-validation score")

X_best_params_KernelRidge=grid_search_KernelRidge.best_params_
X_alpha_KernelRidge = cvres_KernelRidge['param_gamma'].data

plt.plot([X_best_params_KernelRidge.get("gamma"), ] * 2, [0, abs(testscores_mean_KernelRidge[np.where(X_alpha_KernelRidge == X_best_params_KernelRidge.get("gamma"))])],
         linestyle='-.', color='k', marker='x', markeredgewidth=2, ms=8)

plt.annotate("%0.2f" % abs(testscores_mean_KernelRidge[np.where(X_alpha_KernelRidge == X_best_params_KernelRidge.get("gamma"))]),
             (X_best_params_KernelRidge.get("gamma"), abs(testscores_mean_KernelRidge[np.where(X_alpha_KernelRidge == X_best_params_KernelRidge.get("gamma"))]+ 0.005)))

plt.legend(loc="best") 
plt.grid()


In [ ]:
grid_search_KernelRidge.best_params_


In [ ]:
grid_search_KernelRidge.best_estimator_


In [ ]:
for mean_score, params in zip(cvres_KernelRidge["mean_test_score"],cvres_KernelRidge["params"]):
    print(np.sqrt(-mean_score), params)


# Kernel Ridge linear

combina Kernel ridge regression(mínimos cuadrados lineales con la regularización de la norma l2) con el truco del kernel. Así aprende una función lineal en el espacio inducido por el kernel respectivo y los datos. Para los núcleos no lineales, esto corresponde a una función no lineal en el espacio original.


In [ ]:
from sklearn.kernel_ridge import KernelRidge
ridKernelRidge_l =KernelRidge(kernel='linear')
ridKernelRidge_l.fit(housing_prepared[:8500,:],housing_labels[:8500])


In [ ]:
KernelRidge_l = cross_val_score(ridKernelRidge_l , housing_prepared[:8500,:],housing_labels[:8500],
                         scoring="neg_mean_squared_error", cv=5) #scikitlearn trabaja con función util (mayor mejor) no función de costo (menor mejor)
tree_rmse_scores_KernelRidge_l = np.sqrt(-KernelRidge_l)


In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores_KernelRidge_l)


In [ ]:
title = "Curvas de aprendizaje "
plt.figure()
plt.title(title)
plt.xlabel("Training examples")
plt.ylabel("Score")

    
train_sizes_KernelRidge_l, train_scores_KernelRidge_l, test_scores_KernelRidge_l = \
    learning_curve(ridKernelRidge_l, housing_prepared[:8500,:],housing_labels[:8500],
                   scoring="neg_mean_squared_error", cv=5)

train_scores_mean_KernelRidge_l = abs(np.mean(train_scores_KernelRidge_l, axis=1))
train_scores_std_KernelRidge_l = abs(np.std(train_scores_KernelRidge_l, axis=1))
test_scores_mean_KernelRidge_l = abs(np.mean(test_scores_KernelRidge_l, axis=1))
test_scores_std_KernelRidge_l = abs(np.std(test_scores_KernelRidge_l, axis=1))
plt.grid()
plt.fill_between(train_sizes_KernelRidge_l, train_scores_mean_KernelRidge_l - train_scores_std_KernelRidge_l,
                 train_scores_mean_KernelRidge_l + train_scores_std_KernelRidge_l, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes_KernelRidge_l, test_scores_mean_KernelRidge_l - test_scores_std_KernelRidge_l,
                 test_scores_mean_KernelRidge_l + test_scores_std_KernelRidge_l, alpha=0.1, color="g")

plt.plot(train_sizes_KernelRidge_l, train_scores_mean_KernelRidge_l, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes_KernelRidge_l, test_scores_mean_KernelRidge_l, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best") 



# Bayesian Ridge Regression

estima un modelo probabilístico del problema de regresión como se describe anteriormente. El previo para el coeficiente. w Es dado por un gaussiano esférico

In [ ]:
from sklearn.linear_model import BayesianRidge

ridBayesianRidge =BayesianRidge(compute_score=True)
ridBayesianRidge.fit(housing_prepared,housing_labels)



In [ ]:
BayesianRidge = cross_val_score(ridBayesianRidge ,housing_prepared,housing_labels,
                         scoring="neg_mean_squared_error", cv=5) #scikitlearn trabaja con función util (mayor mejor) no función de costo (menor mejor)
tree_rmse_scores_BayesianRidge = np.sqrt(-BayesianRidge)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores_BayesianRidge)

In [ ]:
title = "Curvas de aprendizaje "
plt.figure()
plt.title(title)
plt.xlabel("Training examples")
plt.ylabel("Score")
    
train_sizes_BayesianRidge, train_scores_BayesianRidge, test_scores_BayesianRidge = \
    learning_curve(ridBayesianRidge, housing_prepared,housing_labels,
                   scoring="neg_mean_squared_error", cv=5)

train_scores_mean_BayesianRidge =abs( np.mean(train_scores_BayesianRidge, axis=1))
train_scores_std_BayesianRidge = abs(np.std(train_scores_BayesianRidge, axis=1))
test_scores_mean_BayesianRidge = abs(np.mean(test_scores_BayesianRidge, axis=1))
test_scores_std_BayesianRidge = abs(np.std(test_scores_BayesianRidge, axis=1))
plt.grid()
plt.fill_between(train_sizes_BayesianRidge, train_scores_mean_BayesianRidge - train_scores_std_BayesianRidge,
                 train_scores_mean_BayesianRidge + train_scores_std_BayesianRidge, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes_BayesianRidge, test_scores_mean_BayesianRidge - test_scores_std_BayesianRidge,
                 test_scores_mean_BayesianRidge + test_scores_std_BayesianRidge, alpha=0.1, color="g")

plt.plot(train_sizes_BayesianRidge, train_scores_mean_BayesianRidge, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes_BayesianRidge, test_scores_mean_BayesianRidge, 'o-', color="g",
         label="Cross-validation score")



plt.figure(figsize=(6, 5))
plt.title("Marginal log-likelihood")
plt.plot(ridBayesianRidge.scores_, color='navy', linewidth=2)
plt.ylabel("Score")
plt.xlabel("Iterations")

# Automatic Relevance Determination Regression (ARD)

Ajuste los pesos de un modelo de regresión, utilizando un ARD anterior. Se asume que los pesos del modelo de regresión están en distribuciones gaussianas. También estimar los parámetros lambda (precisiones de las distribuciones de los pesos) y alfa (precisión de la distribución del ruido). La estimación se realiza mediante un procedimiento iterativo (Maximización de la evidencia).

In [ ]:
from sklearn.linear_model import ARDRegression

threshold_lambda = np.array([0.001,10,12,15])
param_grid_ARD = [ {"threshold_lambda": threshold_lambda}]

grid_search_ARD= GridSearchCV(ARDRegression(threshold_lambda=1e5),param_grid_ARD, cv=5,
                                         scoring='neg_mean_squared_error', return_train_score=True)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search_ARD.fit(housing_prepared[:8500,:],housing_labels[:8500])

In [ ]:
cvres_ARD = grid_search_ARD.cv_results_    
testscores_mean_ARD =abs(cvres_ARD["mean_test_score"])
testscores_sd_ARD = abs(cvres_ARD['std_test_score'])

trainscores_mean_ARD =abs(cvres_ARD['mean_train_score'])
trainscores_sd_ARD = abs(cvres_ARD[ 'std_train_score'])

X_axis_ARD= np.array(cvres_ARD['param_threshold_lambda'].data, dtype=float)
plt.fill_between(X_axis_ARD, testscores_mean_ARD- testscores_sd_ARD,
                        testscores_mean_ARD + testscores_sd_ARD,alpha=0.1,color="r")    

plt.plot(X_axis_ARD, testscores_mean_ARD, 'o-', color="r",label="Training score")

plt.fill_between(X_axis_ARD, trainscores_mean_ARD - trainscores_sd_ARD,
                 trainscores_mean_ARD + trainscores_sd_ARD, alpha=0.1, color="g")

plt.plot(X_axis_ARD, trainscores_mean_ARD, 'o-', color="g",
         label="Cross-validation score")

X_best_params_ARD=grid_search_ARD.best_params_
X_alpha_ARD = cvres_ARD['param_threshold_lambda'].data

plt.plot([X_best_params_ARD.get('threshold_lambda'), ] * 2, [0, abs(testscores_mean_ARD[np.where(X_alpha_ARD == X_best_params_ARD.get('threshold_lambda'))])],
         linestyle='-.', color='k', marker='x', markeredgewidth=2, ms=8)

plt.annotate("%0.2f" % abs(testscores_mean_ARD[np.where(X_alpha_ARD== X_best_params_ARD.get('threshold_lambda'))]),
             (X_best_params_ARD.get('threshold_lambda'), abs(testscores_mean_ARD[np.where(X_alpha_ARD == X_best_params_ARD.get('threshold_lambda'))]+ 0.005)))

plt.legend(loc="best") 
plt.grid()
plt.legend(loc="best") 
title = "Curvas de aprendizaje "
plt.title(title)
plt.xlabel("Training examples")
plt.ylabel("Score")

In [ ]:
grid_search_ARD.best_params_

In [ ]:
for mean_score, params in zip(cvres_ARD["mean_test_score"],cvres_ARD["params"]):
    print(np.sqrt(-mean_score), params)

# Seleccionar un modelo Sintonizado con los mejores parámetros

In [ ]:

#linear regression
test_rmse_scores_lin_reg = np.sqrt(test_scores_mean_lin_reg )
#Regresión de Ridge
test_score_ridgeReg=np.array([abs(cvres_ridgeReg['split0_test_score']),abs(cvres_ridgeReg['split1_test_score']),
                              abs(cvres_ridgeReg['split2_test_score']),abs(cvres_ridgeReg['split3_test_score']),
                              abs(cvres_ridgeReg['split4_test_score'])])
params_optimo_ridgeReg=np.where(X_alpha_ridgeReg == X_best_params_ridgeReg.get("alpha"))
test_rmse_scores_ridgeReg=np.sqrt(test_score_ridgeReg[0:,np.int(params_optimo_ridgeReg[-1])])

#Regresión de Lasso
test_score_Lasso=np.array([abs(cvres_ridLasso['split0_test_score']),abs(cvres_ridLasso['split1_test_score']),
                           abs(cvres_ridLasso['split2_test_score']),abs(cvres_ridLasso['split3_test_score']),
                           abs(cvres_ridLasso['split4_test_score'])])
params_optimo_Lassog=np.where(X_alpha_ridLasso == X_best_params_ridLasso.get("alpha"))
test_rmse_scores_Lassog=np.sqrt(test_score_Lasso[0:,np.int(params_optimo_Lassog[-1])])

#Regresión de Elastic
test_score_Elastic=np.array([abs(cvres_ridElastic['split0_test_score']),abs(cvres_ridElastic['split1_test_score']),
                             abs(cvres_ridElastic['split2_test_score']),abs(cvres_ridElastic['split3_test_score']),
                             abs(cvres_ridElastic['split4_test_score'])])



params_optimo_Elastic=np.where(np.array(cvres_ridElastic['params'])==grid_search_ridElastic.best_params_)
test_rmse_scores_Elastic=np.sqrt(test_score_Elastic[0:,np.int(params_optimo_Elastic[-1])])

#KernelRidge rbf
test_score_KernelRidg=np.array([abs(cvres_KernelRidge['split0_test_score']),abs(cvres_KernelRidge['split1_test_score']),
                                abs(cvres_KernelRidge['split2_test_score']),abs(cvres_KernelRidge['split3_test_score']),
                                abs(cvres_KernelRidge['split4_test_score'])])
params_optimo_KernelRidg=np.where(X_alpha_KernelRidge == X_best_params_KernelRidge.get("gamma"))
test_rmse_scores_KernelRidg=np.sqrt(test_score_KernelRidg[0:,np.int(params_optimo_KernelRidg[-1])])

#Kernel Ridge linear
test_rmse_scores_KernelRidge_l = np.sqrt(test_scores_mean_KernelRidge_l )

#Bayesian Ridge Regression
test_rmse_BayesianRidge=np.sqrt(test_scores_mean_BayesianRidge)


#Automatic Relevance Determination Regression (ARD)

test_score_ARD=np.array([abs(cvres_ARD['split0_test_score']),abs(cvres_ARD['split1_test_score']),
                         abs(cvres_ARD['split2_test_score']),abs(cvres_ARD['split3_test_score']),
                         abs(cvres_ARD['split4_test_score'])])

params_optimo_ARD=np.where(X_alpha_ARD == X_best_params_ARD.get('threshold_lambda'))
test_rmse_scores_ARD=np.sqrt(test_score_ARD[0:,np.int(params_optimo_ARD[-1])])



In [ ]:
# Función que convierte los datos en el factor de euros
def modelos_name(name):
    # Convert M in value column to millions
    if name == 1.0:
        return 'LinearRegression'
    if name == 2.0:
        return 'RegresRidge'
    if name == 3.0:
        return 'RegresLasso'
    if name == 4.0:
        return 'Elastic'
    if name == 5.0:
        return 'KernelRidge'
    if name == 6.0:
        return 'KernelRidgel'
    if name == 7.0:
        return 'Bayesian'
    if name == 8.0:
        return 'ARD'
    if name == 9.0:
        return 'Forest'


#modelos
modelos=np.array([[np.ones(len(test_rmse_scores_lin_reg))], [np.ones(len(test_rmse_scores_ridgeReg))*2], 
                  [np.ones(len(test_rmse_scores_Lassog))*3],[np.ones(len(test_rmse_scores_Elastic))*4],
                  [np.ones(len(test_rmse_scores_KernelRidg))*5],[np.ones(len(test_rmse_scores_KernelRidge_l))*6],
                  [np.ones(len(test_rmse_scores_ARD))*7],[np.ones(len(test_rmse_scores_ARD))*8],
                  [np.ones(len(test_rmse_scores_forest_reg))*9]
                 ])

modelos=np.reshape(modelos,45)
parametros_modelos=np.array([[test_rmse_scores_lin_reg], [test_rmse_scores_ridgeReg], 
                             [test_rmse_scores_Lassog], [test_rmse_scores_Elastic],
                             [test_rmse_scores_KernelRidg],[test_rmse_scores_KernelRidge_l],
                             [test_rmse_BayesianRidge],[test_rmse_scores_ARD],
                             [test_rmse_scores_forest_reg]
                            ])
parametros_modelos=np.reshape(parametros_modelos, 45)

rultados_de_test_train= {'modelos': pd.Series(modelos),                         
                         'parametros_modelos': pd.Series(parametros_modelos)}
                                                         

rultados_de_test_train_table = pd.DataFrame(rultados_de_test_train)
rultados_de_test_train_table['modelos']= rultados_de_test_train_table['modelos'].apply(lambda x:  modelos_name(x) )

plt.figure(figsize=(25,10))
sns.boxenplot(rultados_de_test_train_table['modelos'],rultados_de_test_train_table['parametros_modelos'])



El intervalo de confianza al 95% se puede calcular para el RMSE sobre el conjunto de test:

We could compute the interval manually like this:

# Material extra

## Pipeline completo para preparación de datos y predicción

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(housing, housing_labels)
full_pipeline_with_predictor.predict(some_data)

## Creación de modelo eficiente con joblib

In [ ]:
my_model = full_pipeline_with_predictor

In [ ]:
from sklearn.externals import joblib #https://joblib.readthedocs.io/en/latest/
joblib.dump(my_model, "my_model.pkl") # DIFF  https://docs.python.org/2/library/pickle.html
#...
my_model_loaded = joblib.load("my_model.pkl") # DIFF

## Ejemplos de distribuciones en SciPy para `RandomizedSearchCV`

In [ ]:
from scipy.stats import geom, expon
geom_distrib=geom(0.5).rvs(10000, random_state=42)
expon_distrib=expon(scale=1).rvs(10000, random_state=42)
plt.hist(geom_distrib, bins=50)
plt.show()
plt.hist(expon_distrib, bins=50)
plt.show()